In [169]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.formula.api as smf

In [170]:
import pandas_profiling

In [171]:
df = pd.read_csv('bankloans.csv')

In [172]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 850 entries, 0 to 849
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       850 non-null    int64  
 1   ed        850 non-null    int64  
 2   employ    850 non-null    int64  
 3   address   850 non-null    int64  
 4   income    850 non-null    int64  
 5   debtinc   850 non-null    float64
 6   creddebt  850 non-null    float64
 7   othdebt   850 non-null    float64
 8   default   700 non-null    float64
dtypes: float64(4), int64(5)
memory usage: 59.9 KB


In [173]:
existing = df[-df.default.isna()]
new = df[df.default.isna()]

In [174]:
existing

,age,ed,employ,address,income,debtinc,creddebt,othdebt,default
0,41,3,17,12,176,9.3,11.359392,5.008608,1.0
1,27,1,10,6,31,17.3,1.362202,4.000798,0.0
2,40,1,15,14,55,5.5,0.856075,2.168925,0.0
3,41,1,15,14,120,2.9,2.658720,0.821280,0.0
4,24,2,2,0,28,17.3,1.787436,3.056564,1.0
...,...,...,...,...,...,...,...,...,...
695,36,2,6,15,27,4.6,0.262062,0.979938,1.0
696,29,2,6,4,21,11.5,0.369495,2.045505,0.0
697,33,1,15,3,32,7.6,0.491264,1.940736,0.0
698,45,1,19,22,77,8.4,2.302608,4.165392,0.0


In [175]:
existing

,age,ed,employ,address,income,debtinc,creddebt,othdebt,default
0,41,3,17,12,176,9.3,11.359392,5.008608,1.0
1,27,1,10,6,31,17.3,1.362202,4.000798,0.0
2,40,1,15,14,55,5.5,0.856075,2.168925,0.0
3,41,1,15,14,120,2.9,2.658720,0.821280,0.0
4,24,2,2,0,28,17.3,1.787436,3.056564,1.0
...,...,...,...,...,...,...,...,...,...
695,36,2,6,15,27,4.6,0.262062,0.979938,1.0
696,29,2,6,4,21,11.5,0.369495,2.045505,0.0
697,33,1,15,3,32,7.6,0.491264,1.940736,0.0
698,45,1,19,22,77,8.4,2.302608,4.165392,0.0


In [176]:
reprot = pandas_profiling.ProfileReport(existing)
reprot.to_file('report.html')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [177]:
#Missing vlaue treatment
def missings_treat(x):
    x = x.fillna(x.median())
    return x

#Handling Outliers - Method2
def outlier_capping(x):
    x = x.clip(upper = x.quantile(0.99), lower =x.quantile(0.01) )
    return x

existing=existing.apply(lambda x: missings_treat(x))
existing=existing.apply(lambda x: outlier_capping(x))

In [178]:
pd.qcut?

In [179]:
existing.columns

Index(['age', 'ed', 'employ', 'address', 'income', 'debtinc', 'creddebt',
       'othdebt', 'default'],
      dtype='object')

In [180]:
#Assumptions

#WOE or Information calculation

existing['age_bins'] =pd.qcut(existing.age, q=5, labels=False)
existing['employ_bins'] =pd.qcut(existing.employ, q=5, labels=False)
existing['address_bins'] =pd.qcut(existing.address, q=5, labels=False)
existing['income_bins'] =pd.qcut(existing.income, q=5, labels=False)
existing['debtinc_bins'] =pd.qcut(existing.debtinc, q=5, labels=False)
existing['creddebt_bins'] =pd.qcut(existing.creddebt, q=5, labels=False)
existing['othdebt_bins'] =pd.qcut(existing.othdebt, q=5, labels=False)

In [181]:
existing.columns

Index(['age', 'ed', 'employ', 'address', 'income', 'debtinc', 'creddebt',
       'othdebt', 'default', 'age_bins', 'employ_bins', 'address_bins',
       'income_bins', 'debtinc_bins', 'creddebt_bins', 'othdebt_bins'],
      dtype='object')

In [182]:
existing1 = existing[['ed', 'age_bins', 'employ_bins', 'address_bins',
       'income_bins', 'debtinc_bins', 'creddebt_bins', 'othdebt_bins', 'default']]

In [183]:
#Information value calculation
def calculate_woe_iv(dataset, feature, target):
    lst = []
    for i in range(dataset[feature].nunique()):
        val = list(dataset[feature].unique())[i]
        lst.append({
            'Value': val,
            'All': dataset[dataset[feature] == val].count()[feature],
            'Good': dataset[(dataset[feature] == val) & (dataset[target] == 0)].count()[feature],
            'Bad': dataset[(dataset[feature] == val) & (dataset[target] == 1)].count()[feature]
        })
        
    dset = pd.DataFrame(lst)
    dset['Distr_Good'] = dset['Good'] / dset['Good'].sum()
    dset['Distr_Bad'] = dset['Bad'] / dset['Bad'].sum()
    dset['WoE'] = np.log(dset['Distr_Good'] / dset['Distr_Bad'])
    dset = dset.replace({'WoE': {np.inf: 0, -np.inf: 0}})
    dset['IV'] = (dset['Distr_Good'] - dset['Distr_Bad']) * dset['WoE']
    iv = dset['IV'].sum()
    
    dset = dset.sort_values(by='WoE')
    
    return dset, iv

In [184]:
for col in existing1.columns:
    if col == 'default': continue
    else:
        print('WoE and IV for column: {}'.format(col))
        df, iv = calculate_woe_iv(existing1, col, 'default')
        print(df)
        print('IV score: {:.2f}'.format(iv))
        print('\n')

WoE and IV for column: ed
   Value  All  Good  Bad  Distr_Good  Distr_Bad       WoE        IV
3    4.0   43    28   15    0.054159   0.081967 -0.414402  0.011524
0    3.0   87    57   30    0.110251   0.163934 -0.396703  0.021296
2    2.0  198   139   59    0.268859   0.322404 -0.181620  0.009725
1    1.0  372   293   79    0.566731   0.431694  0.272168  0.036753
IV score: 0.08


WoE and IV for column: age_bins
   Value  All  Good  Bad  Distr_Good  Distr_Bad       WoE        IV
1      0  174   106   68    0.205029   0.371585 -0.594625  0.099038
4      1  129    92   37    0.177950   0.202186 -0.127686  0.003095
2      4  140   112   28    0.216634   0.153005  0.347738  0.022126
3      2  136   109   27    0.210832   0.147541  0.356954  0.022592
0      3  121    98   23    0.189555   0.125683  0.410917  0.026246
IV score: 0.17


WoE and IV for column: employ_bins
   Value  All  Good  Bad  Distr_Good  Distr_Bad       WoE        IV
2      0  155    79   76    0.152805   0.415301 -0.999842

In [185]:
IV_vars =  ['debtinc',  'employ', 'creddebt', 'age', 'income', 'address']

In [186]:
existing

,age,ed,employ,address,income,debtinc,creddebt,othdebt,default,age_bins,employ_bins,address_bins,income_bins,debtinc_bins,creddebt_bins,othdebt_bins
0,41.0,3.0,17.0,12.0,176.0,9.3,9.891428,5.008608,1.0,3,4,3,4,2,4,4
1,27.0,1.0,10.0,6.0,31.0,17.3,1.362202,4.000798,0.0,0,3,2,2,4,3,3
2,40.0,1.0,15.0,14.0,55.0,5.5,0.856075,2.168925,0.0,3,4,3,3,1,2,2
3,41.0,1.0,15.0,14.0,120.0,2.9,2.658720,0.821280,0.0,3,4,3,4,0,4,0
4,24.0,2.0,2.0,0.0,28.0,17.3,1.787436,3.056564,1.0,0,0,0,1,4,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,36.0,2.0,6.0,15.0,27.0,4.6,0.262062,0.979938,1.0,2,2,4,1,1,0,1
696,29.0,2.0,6.0,4.0,21.0,11.5,0.369495,2.045505,0.0,1,2,1,0,3,1,2
697,33.0,1.0,15.0,3.0,32.0,7.6,0.491264,1.940736,0.0,2,4,1,2,2,1,2
698,45.0,1.0,19.0,22.0,77.0,8.4,2.302608,4.165392,0.0,4,4,4,4,2,4,3


In [187]:
# Univariate regression



In [188]:
import sklearn.metrics as metrics

In [189]:
m1 = smf.logit('default~debtinc', data=existing).fit()
metrics.roc_auc_score(existing.default, m1.predict())

Optimization terminated successfully.
         Current function value: 0.501389
         Iterations 6


0.7352527718764202

In [190]:
auc_df = pd.DataFrame()
for col in existing.columns.difference(['default']):
    model = smf.logit('default~'+str(col), data=existing).fit()
    auc = metrics.roc_auc_score(existing.default, model.predict(existing))
    temp = pd.DataFrame([col, auc]).T
    auc_df = pd.concat([auc_df, temp], axis=0)

Optimization terminated successfully.
         Current function value: 0.559856
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.562831
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.564673
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.560737
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.548956
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.557392
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.501389
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.508991
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.567774
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.527166
  

In [191]:
auc_df.columns = ['feature', 'auc']

In [192]:
auc_df.sort_values(by='auc', inplace=True, ascending=False)

In [193]:
auc_df

,feature,auc
0,debtinc,0.735253
0,debtinc_bins,0.726644
0,employ_bins,0.705679
0,employ,0.704569
0,creddebt,0.636189
0,creddebt_bins,0.618379
0,address,0.611192
0,age_bins,0.607429
0,age,0.603165
0,income,0.600136


In [194]:
X_new = existing[['debtinc',  'employ', 'creddebt',   'address']]  #dropped variables due to multicollinerity - 'age', 'income',

In [195]:
X_new

,debtinc,employ,creddebt,address
0,9.3,17.0,9.891428,12.0
1,17.3,10.0,1.362202,6.0
2,5.5,15.0,0.856075,14.0
3,2.9,15.0,2.658720,14.0
4,17.3,2.0,1.787436,0.0
...,...,...,...,...
695,4.6,6.0,0.262062,15.0
696,11.5,6.0,0.369495,4.0
697,7.6,15.0,0.491264,3.0
698,8.4,19.0,2.302608,22.0


In [196]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [197]:
### VIF Calculation for variables
vif = pd.DataFrame()
vif["VIF_Factor"] = [variance_inflation_factor(X_new.values, i) for i in range(X_new.shape[1])]
vif["features"] = X_new.columns

In [198]:
vif.sort_values(by='VIF_Factor', ascending=False)

,VIF_Factor,features
2,2.880403,creddebt
1,2.813053,employ
0,2.788343,debtinc
3,2.481497,address


In [199]:
Final_vars = ['creddebt', 'employ', 'debtinc', 'address', 'default']
existing_final = existing[Final_vars]

In [200]:
from sklearn.model_selection import train_test_split

In [201]:
train, test = train_test_split(existing_final, test_size=0.3, random_state=123)

In [202]:
# building model

model_logit = smf.logit('default~creddebt+employ+debtinc+address', data=train).fit()

Optimization terminated successfully.
         Current function value: 0.400953
         Iterations 7


In [203]:
print(model_logit.summary())

                           Logit Regression Results                           
Dep. Variable:                default   No. Observations:                  490
Model:                          Logit   Df Residuals:                      485
Method:                           MLE   Df Model:                            4
Date:                Tue, 07 Jun 2022   Pseudo R-squ.:                  0.3118
Time:                        12:55:11   Log-Likelihood:                -196.47
converged:                       True   LL-Null:                       -285.50
Covariance Type:            nonrobust   LLR p-value:                 1.944e-37
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.8974      0.299     -3.000      0.003      -1.484      -0.311
creddebt       0.6506      0.106      6.146      0.000       0.443       0.858
employ        -0.2461      0.033     -7.443      0.0

In [204]:
train.head(2)

,creddebt,employ,debtinc,address,default
404,2.099636,18.0,15.4,10.0,0.0
63,2.159850,4.0,7.0,9.0,0.0


In [205]:
#mathematical equation:
#logit(Y) = log(p/1-p) = 0.6506*creddebt - 0.2461*employ +0.0870*debtinc-0.0724*address-0.8974
# P(Y=1) = exp(LE)/(1+exp(LE))

In [206]:
creddebt = 2.099636
employ = 18
debtinc = 15.4
address = 10

In [207]:
print(np.exp(0.6506*creddebt - 0.2461*employ +0.0870*debtinc-0.0724*address-0.8974)/(1+np.exp(0.6506*creddebt - 0.2461*employ +0.0870*debtinc-0.0724*address-0.8974)))

0.034046882834027736


In [208]:
print('train_auc=', metrics.roc_auc_score(train.default, model_logit.predict(train)))
print('test_auc=', metrics.roc_auc_score(test.default, model_logit.predict(test)))

train_auc= 0.8562510580667005
test_auc= 0.8403009002343076


In [209]:
print('train_gini=', 2*metrics.roc_auc_score(train.default, model_logit.predict(train))-1)
print('test_gini=', 2*metrics.roc_auc_score(test.default, model_logit.predict(test))-1)

train_gini= 0.7125021161334011
test_gini= 0.6806018004686152


In [210]:
#how to identify best cut-off
# %1's in Y
# using sensitivity+specificity
# Decile analysis

In [211]:
train.default.mean()

0.2693877551020408

In [212]:
np.linspace(0,1, num=100)

array([0.        , 0.01010101, 0.02020202, 0.03030303, 0.04040404,
       0.05050505, 0.06060606, 0.07070707, 0.08080808, 0.09090909,
       0.1010101 , 0.11111111, 0.12121212, 0.13131313, 0.14141414,
       0.15151515, 0.16161616, 0.17171717, 0.18181818, 0.19191919,
       0.2020202 , 0.21212121, 0.22222222, 0.23232323, 0.24242424,
       0.25252525, 0.26262626, 0.27272727, 0.28282828, 0.29292929,
       0.3030303 , 0.31313131, 0.32323232, 0.33333333, 0.34343434,
       0.35353535, 0.36363636, 0.37373737, 0.38383838, 0.39393939,
       0.4040404 , 0.41414141, 0.42424242, 0.43434343, 0.44444444,
       0.45454545, 0.46464646, 0.47474747, 0.48484848, 0.49494949,
       0.50505051, 0.51515152, 0.52525253, 0.53535354, 0.54545455,
       0.55555556, 0.56565657, 0.57575758, 0.58585859, 0.5959596 ,
       0.60606061, 0.61616162, 0.62626263, 0.63636364, 0.64646465,
       0.65656566, 0.66666667, 0.67676768, 0.68686869, 0.6969697 ,
       0.70707071, 0.71717172, 0.72727273, 0.73737374, 0.74747

In [213]:
train['prob'] = model_logit.predict(train)
test['prob'] = model_logit.predict(test)

In [214]:
train.default

404    0.0
63     0.0
34     0.0
33     0.0
583    1.0
      ... 
98     0.0
322    0.0
382    1.0
365    0.0
510    0.0
Name: default, Length: 490, dtype: float64

In [215]:
metrics.confusion_matrix(train.default, np.where(train.prob>0.27,1,0))

array([[274,  84],
       [ 27, 105]], dtype=int64)

In [216]:
roc_df = pd.DataFrame()
for cut_off in np.linspace(0,1, num=100):
    TN = metrics.confusion_matrix(train.default, np.where(train.prob>cut_off,1,0))[0][0]
    TP = metrics.confusion_matrix(train.default, np.where(train.prob>cut_off,1,0))[1][1]
    FN = metrics.confusion_matrix(train.default, np.where(train.prob>cut_off,1,0))[1][0]
    FP = metrics.confusion_matrix(train.default, np.where(train.prob>cut_off,1,0))[0][1]
    sensitivity = TP/(FN+TP)
    specificity = TN/(TN+FP)
    accuracy = (TP+TN)/(TP+TN+FN+FP)
    total = sensitivity+ specificity
    temp_df = pd.DataFrame([cut_off, sensitivity, specificity, accuracy,total]).T
    roc_df = pd.concat([roc_df, temp_df], axis=0)

In [217]:
roc_df.columns =['cut_off', 'sensitivity', 'specificity', 'accuracy','total']

In [218]:
roc_df.sort_values(by='total', ascending=False)

,cut_off,sensitivity,specificity,accuracy,total
0,0.272727,0.787879,0.773743,0.777551,1.561622
0,0.262626,0.803030,0.748603,0.763265,1.551634
0,0.252525,0.810606,0.737430,0.757143,1.548036
0,0.181818,0.909091,0.631285,0.706122,1.540376
0,0.232323,0.825758,0.712291,0.742857,1.538048
...,...,...,...,...,...
0,0.969697,0.030303,1.000000,0.738776,1.030303
0,0.979798,0.022727,1.000000,0.736735,1.022727
0,0.989899,0.000000,1.000000,0.730612,1.000000
0,0.000000,1.000000,0.000000,0.269388,1.000000


In [219]:
best_cutoff = roc_df.sort_values(by='total', ascending=False).head(1)['cut_off'][0]

In [220]:
train['pred'] = np.where(train.prob>best_cutoff, 1, 0)
test['pred'] = np.where(test.prob>best_cutoff, 1, 0)

In [221]:
test

,creddebt,employ,debtinc,address,default,prob,pred
57,1.061496,16.0,2.4,13.0,0.0,0.007564,0
478,0.820870,27.0,11.5,21.0,0.0,0.000538,0
597,2.594144,5.0,14.8,0.0,1.0,0.700041,1
689,0.227268,4.0,10.7,2.0,0.0,0.279323,1
647,0.882833,6.0,4.9,6.0,0.0,0.140905,0
...,...,...,...,...,...,...,...
41,1.807760,17.0,8.0,3.0,0.0,0.031496,0
617,0.449820,6.0,5.1,7.0,1.0,0.104843,0
226,3.703260,10.0,18.1,8.0,0.0,0.511585,1
26,0.707319,9.0,6.7,26.0,0.0,0.018850,0


In [222]:
metrics.confusion_matrix(train.default, train.pred)

array([[277,  81],
       [ 28, 104]], dtype=int64)

In [223]:
print(metrics.classification_report(train.default, train.pred))

              precision    recall  f1-score   support

         0.0       0.91      0.77      0.84       358
         1.0       0.56      0.79      0.66       132

    accuracy                           0.78       490
   macro avg       0.74      0.78      0.75       490
weighted avg       0.81      0.78      0.79       490



In [224]:
print(metrics.classification_report(test.default, test.pred))

              precision    recall  f1-score   support

         0.0       0.91      0.77      0.84       159
         1.0       0.52      0.76      0.62        51

    accuracy                           0.77       210
   macro avg       0.72      0.77      0.73       210
weighted avg       0.82      0.77      0.78       210



In [225]:
#convert probability of default into credit score
#score = 1000 - prob*1000

train['cred_score'] = 1000-1000*train.prob

In [226]:
test['cred_score'] = 1000-1000*test.prob

In [227]:
train.head()

,creddebt,employ,debtinc,address,default,prob,pred,cred_score
404,2.099636,18.0,15.400,10.0,0.0,0.034050,0,965.949831
63,2.159850,4.0,7.000,9.0,0.0,0.372982,1,627.017684
34,0.077140,8.0,2.900,1.0,0.0,0.066860,0,933.140133
33,1.018656,8.0,14.400,4.0,0.0,0.224398,0,775.602357
583,7.320000,16.0,30.701,14.0,1.0,0.829851,1,170.148675


In [228]:
cred_score_cutoff = 1000-best_cutoff*1000

In [229]:
cred_score_cutoff

727.2727272727273

In [230]:
import pickle 

In [231]:
with open("model.pkl", 'wb') as f:
    pickle.dump('model_logit', f)

In [232]:
#Decile Analysis - How good is your model, model validation, finding the cut-off, other business related decision can be taken

In [233]:
train['decile'] =pd.qcut(train.prob, 10, labels=False)
test['decile'] =pd.qcut(test.prob, 10, labels=False)

In [234]:
train

,creddebt,employ,debtinc,address,default,prob,pred,cred_score,decile
404,2.099636,18.0,15.400,10.0,0.0,0.034050,0,965.949831,1
63,2.159850,4.0,7.000,9.0,0.0,0.372982,1,627.017684,7
34,0.077140,8.0,2.900,1.0,0.0,0.066860,0,933.140133,2
33,1.018656,8.0,14.400,4.0,0.0,0.224398,0,775.602357,5
583,7.320000,16.0,30.701,14.0,1.0,0.829851,1,170.148675,9
...,...,...,...,...,...,...,...,...,...
98,0.681093,4.0,6.300,9.0,0.0,0.176189,0,823.811278,4
322,0.431200,16.0,1.600,22.0,0.0,0.002452,0,997.547841,0
382,4.003840,2.0,16.000,12.0,1.0,0.850441,1,149.558698,9
365,0.428640,3.0,1.900,13.0,0.0,0.105924,0,894.076246,3


In [235]:
train['goods'] = 1-train.default
test['goods'] = 1-test.default

In [236]:
decile_results_train = train.groupby(['decile']).agg(min_prob = ('prob', 'min'),
                              max_prob = ('prob', 'max'),
                              No_bads = ('default', 'sum'),
                              No_goods = ('goods', 'sum'), 
                              total = ('default', 'count'))

decile_results_test = test.groupby(['decile']).agg(min_prob = ('prob', 'min'),
                              max_prob = ('prob', 'max'),
                              No_bads = ('default', 'sum'),
                              No_goods = ('goods', 'sum'), 
                              total = ('default', 'count'),)

In [239]:
decile_results_test

,min_prob,max_prob,No_bads,No_goods,total
decile,,,,,
0,0.000538,0.009694,0.0,21.0,21
1,0.009737,0.031551,1.0,20.0,21
2,0.032405,0.068534,2.0,19.0,21
3,0.074101,0.123145,1.0,20.0,21
4,0.125939,0.164836,3.0,18.0,21
5,0.165733,0.233115,2.0,19.0,21
6,0.233750,0.334842,7.0,14.0,21
7,0.336724,0.439296,10.0,11.0,21
8,0.445562,0.637355,8.0,13.0,21


In [240]:
decile_results_train.to_csv('train_deciles.csv')
decile_results_test.to_csv('test_deciles.csv')

In [243]:
new_df = new[['address','employ',  'debtinc', 'creddebt']]

In [245]:
new['prob'] = model_logit.predict(new_df)

In [246]:
new['cred_score'] = 1000-new.prob*1000

In [251]:
best_cutoff

0.27272727272727276

In [247]:
new['pred'] = np.where(new.prob>best_cutoff, 1, 0)
#new['pred']  = np.where(new.cred_score<(1-best_cutoff)*1000, 1, 0)

In [252]:
new['seg'] = np.where(new.prob>0.508,"High Risk", np.where(new.prob>0.19254,"Medium Risk", "Low Risk"))

In [253]:
new.seg.value_counts()

Low Risk       82
Medium Risk    44
High Risk      24
Name: seg, dtype: int64